In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [157]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [158]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [159]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [160]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [161]:
#handling dates in days
users.created_date=pd.to_datetime(users.created_date)
#users['today']=np.datetime64('2019-06-01')
users['account_duration'] = (np.datetime64('2019-06-01') - users['created_date']).dt.days

In [162]:
#users['month']=pd.DatetimeIndex(users['created_date']).month
#users['year']=pd.DatetimeIndex(users['created_date']).year
#users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [163]:
amount_of_completed_trans=trans[trans['transactions_state']=='COMPLETED'].groupby(['user_id'])['transactions_state'].count()
amount_of_not_completed_trans=trans[trans['transactions_state']!='COMPLETED'].groupby(['user_id'])['transactions_state'].count()

In [164]:
compl_not_compl_analogy=amount_of_completed_trans/amount_of_not_completed_trans

In [165]:
users=pd.merge(users , compl_not_compl_analogy, how='left', left_on='user_id', right_index=True)

In [166]:
#tempo table me hmeromhnies twn transaction 
#Oi hmeromhnies ypologizontai se meres apo thn 1/6
tempo=trans[['user_id','created_date']]
tempo.sort_values(['user_id','created_date'],inplace=True)
tempo['created_date']=pd.to_datetime(tempo['created_date'])
tempo['days_from_transaction']=(np.datetime64('2019-06-01') - tempo['created_date']).dt.days
tempo.drop(columns=['created_date'], inplace=True)


c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
c:\users\thrash\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [167]:
#uni table exei tis unique dates poy exei kanei ka8e user transaction
uni = tempo.drop_duplicates(["user_id","days_from_transaction"])


In [168]:
#ypologizontai o mean tvn diaforwn kathe transaction, kai h max diafora transaction gia kathe user
mean_period_within_trans=uni.groupby(['user_id']).agg(mean_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).mean()))
max_period_within_trans=uni.groupby(['user_id']).agg(max_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).max()))
       

In [169]:
users=pd.merge(users , mean_period_within_trans, how='left', left_on='user_id', right_index=True)

In [170]:
users=pd.merge(users , max_period_within_trans, how='left', left_on='user_id', right_index=True)

In [171]:
#ypologizetai poses unique dates exei kanei transactions kathe user
unique_days_of_trans=tempo.groupby(['user_id']).agg(unique_days_of_transactions=('days_from_transaction', 'nunique'))

In [172]:
users=pd.merge(users , unique_days_of_trans, how='left', left_on='user_id', right_index=True)

In [173]:
#ypolofizetai to std, var twn dates of transactions gia ka8e user
std_var_trans=tempo.groupby('user_id').agg(std_of_transactions=('days_from_transaction','std'),var_of_transactions=('days_from_transaction','var'))


In [174]:
users=pd.merge(users , std_var_trans, how='left', left_on='user_id', right_index=True)

In [175]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [176]:
#number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
#number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
#users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [177]:
usd_per_user = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user.columns=['amount_of_transactions', 'min_usd', 'max_usd', 'mean_usd', 'sum_usd']
users=pd.merge(users , usd_per_user, how='left', left_on='user_id', right_index=True)

In [178]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [179]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [180]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [181]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

In [182]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

In [183]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [184]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [185]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [186]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [187]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [188]:
#handling dates in days
lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
#lasttrans['today']=np.datetime64('2019-06-01')
lasttrans['last_transaction'] = (np.datetime64('2019-06-01') - lasttrans['created_date']).dt.days

In [189]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [190]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [191]:
trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
firsttrans = trans_sorted_asc.drop_duplicates('user_id').sort_index()
firsttrans=firsttrans[['user_id','created_date']]

In [192]:
#handling dates in days
firsttrans['created_date']=pd.to_datetime(firsttrans['created_date'])
#firsttrans['today']=np.datetime64('2019-06-01')
firsttrans['first_transaction'] = (np.datetime64('2019-06-01')-firsttrans['created_date']).dt.days

In [193]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [194]:
users=pd.merge(users , firsttrans, how='left', left_on='user_id', right_on='user_id')

In [196]:
users['engagement_duration']=users['first_transaction'] - users['last_transaction'] + 1

In [197]:
users['days_passed_for_first_transaction']= users['account_duration']- users['first_transaction'] 

In [199]:
users['transactions_frequency']=users['amount_of_transactions'] / users['engagement_duration']

#Notifications
________________________________________________________


In [204]:
notif_sent=notif[notif['status']=='SENT']
notif_sent=notif[(notif['reason']!='SILVER_ENGAGEMENT_FEES_SAVED') & (notif['reason']!='SILVER_ENGAGEMENT_INACTIVE_CARD')]

In [219]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=notif_sent['reason'])



In [220]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [ ]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [ ]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [ ]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [ ]:
#users[users['user_id']=='user_12544']

In [ ]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [ ]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [ ]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [ ]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [222]:
#dropping columns we don't need
data=users.drop(columns=['user_id','birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','num_referrals','num_successful_referrals','created_date_y','created_date'])


#'year_x','month_x','month_y','year_y','month','year'

In [225]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [226]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [227]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [228]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [230]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
datatest.to_csv('friday_testing.csv',index=False, sep=';')
data.to_csv('friday_users.csv',index=False, sep=';')
users.to_csv('friday_users_feat.csv',index=False, sep=';')

In [ ]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values